In [ ]:
import json
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from adjustText import adjust_text
import scipy.stats as stf
from sklearn.metrics import r2_score

In [ ]:
# Configurazione opzioni Pandas e Matplotlib

pd.set_option('display.max_columns', None)

plt.style.use('ggplot')

In [ ]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_work_vaccinazioni_covid = config_data['path_work_vaccinazioni_covid']
file_path_work_andamento_prezzi_consumo = config_data['path_work_andamento_prezzi_consumo']
file_path_work_risultati_delle_elezioni_legislative_1_turno = config_data['path_work_risultati_elezioni_legislative_1_turno']
file_path_work_risultati_delle_elezioni_legislative_2_turno = config_data['path_work_risultati_elezioni_legislative_2_turno']
file_path_work_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_work_risultati_elezioni_presidenziali_1_turno']
file_path_work_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_work_risultati_elezioni_presidenziali_2_turno']
file_path_work_dati_sulla_disoccupazione = config_data['path_work_dati_sulla_disoccupazione']
file_path_work_andamento_dei_prezzi_dei_carburanti = config_data['path_work_andamento_dei_prezzi_dei_carburanti']
file_path_work_reddito_per_dipartimenti = config_data['path_work_reddito_per_dipartimenti']
file_path_work_inflazione = config_data['path_work_inflazione']

In [ ]:
# Leggo i csv puliti e li mantengo in DataFrame di Pandas

df_risultati_elezioni_presidenziali_1_turno = pd.read_csv(file_path_work_risultati_delle_elezioni_presidenziali_1_turno,
                                                          dtype={'codice_dipartimento': str, 'codice_circoscrizione': str,
                                                                 'codice_comune': str, 'codice_seggio_elettorale': str,
                                                                 'codice_EPCI': str, 'espressi': int, '% voti/espressi': float})
df_risultati_elezioni_presidenziali_2_turno = pd.read_csv(file_path_work_risultati_delle_elezioni_presidenziali_2_turno,
                                                          dtype={'codice_dipartimento': str, 'codice_circoscrizione': str,
                                                                 'codice_comune': str, 'codice_seggio_elettorale': str,
                                                                 'codice_EPCI': str, 'espressi': int, '% voti/espressi': float})
df_risultati_elezioni_legislative_1_turno = pd.read_csv(file_path_work_risultati_delle_elezioni_legislative_1_turno,
                                                        dtype={'codice_dipartimento': str, 'codice_seggio_elettorale': str,
                                                               'codice_comune': str, 'espressi': int})
df_disoccupazione = pd.read_csv(file_path_work_dati_sulla_disoccupazione)
df_reddito_dipartimenti = pd.read_csv(file_path_work_reddito_per_dipartimenti)
df_vaccinazioni_covid = pd.read_csv(file_path_work_vaccinazioni_covid, dtype={'codice_dipartimento': str})
df_inflazione = pd.read_csv(file_path_work_inflazione)
df_prezzi_carburanti = pd.read_csv(file_path_work_andamento_dei_prezzi_dei_carburanti, dtype={'codice_dipartimento': str, 'codice_comune': str})

In [ ]:
# Definizione funzioni ausiliarie (Query)

def percentage_calculator(row):
    if(row['espressi'] == 0):
        return 0
    else:
        return row['voti']/row['espressi']

    
def get_dipartimenti_min_max_perc_voti(df_dataset, cognome=None, min=False):
    """
        Funzione che calcola la maggiore o la minore percentuale di voti per un dato candidato
    """
    
    # raggruppo i dati sulle colonne 'nome_dipartimento' e 'cognome' e ottiengo la somma dei valori della colonna 'voti' e della colonna 'espressi'
    somma_voti_espressi = df_dataset.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
    
    # filtro per cognome
    somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == cognome]
    
    # calcolo la percentuale e ordino in maniera decrescente
    somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
    percentuali = somma_voti_espressi.sort_values(by="% voti_a_favore/voti_totali",ascending=False)
    percentuali.reset_index(drop=True, inplace=True)
    
    # seleziono in base al valore di min i dipartimenti con percentuale maggiore o minore
    if min:
        df_query = percentuali[-20:]
    else:
        df_query = percentuali[:20]
    #display(df_query)
    lista_dipartimenti = df_query["nome_dipartimento"].values.tolist()
    display(lista_dipartimenti)
    return lista_dipartimenti


def differenza_presidenziali_legislative(candidati, partito):
    """
        Funzione che calcola la differenza in valore assoluto tra i voti espressi per uno o più candidati alle presidenziali 
        e i voti espressi per il relativo partito alle legislative
    """

    # filtra i dataframe sui candidati/partito di interesse
    df_presidenziali_candidati = df_risultati_elezioni_presidenziali_1_turno[df_risultati_elezioni_presidenziali_1_turno['cognome'].isin(candidati)]
    df_legislative_candidati = df_risultati_elezioni_legislative_1_turno[df_risultati_elezioni_legislative_1_turno['partito'] == partito]

    # somma i voti espressi per ogni dipartimento
    n_voti_presidenziali = df_presidenziali_candidati.groupby('nome_dipartimento')['espressi'].sum()
    n_voti_legislative = df_legislative_candidati.groupby('nome_dipartimento')['espressi'].sum()

    # merge dei due dataframe in base al dipartimento
    df_merged = pd.merge(n_voti_presidenziali, n_voti_legislative, on='nome_dipartimento', suffixes=('_presidenziali', '_legislative'))
    # aggiunge una colonna al dataset con il calcolo della differenza voti
    df_merged['differenza_voti'] = abs(df_merged['espressi_presidenziali'] - df_merged['espressi_legislative'])
    # ordina il dataframe per differenza voti decrescente
    df_merged = df_merged.sort_values(by='differenza_voti', ascending=False)
    df_merged.reset_index(inplace=True)
    #display(df_merged)

    # ottengo e stampo i 10 dipartimenti con differenza voti maggiore
    lista_dipartimenti = df_merged['nome_dipartimento'].values.tolist()[:10]
    display(lista_dipartimenti)
    return lista_dipartimenti

In [ ]:
# Definizione funzioni ausiliarie (Correlazioni)

def correlation_calculator(x, y, x_label, y_label, etichette_dipartimenti=False):
    """
        Funzione che calcola i coefficienti di correlazione per i vettori di dati inseriti,
        e disegna il grafico dei dati e della loro regressione lineare
    """
    
    slope, intercept, r, p, stderr = st.linregress(x, y)
    line = f'Retta di regressione: y = {intercept:.2f} + {slope:.2f}x'
    if (etichette_dipartimenti):
        fig, ax = plt.subplots(figsize=(16, 10))
    else:
        fig, ax = plt.subplots()
    ax.plot(x, y, linewidth=0, marker='o', label='Dati')
    ax.plot(x, intercept + slope * x, label=line)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.xaxis.label.set_color('black')
    ax.yaxis.label.set_color('black')
    ax.legend(facecolor='white')

    if (etichette_dipartimenti):
        # Aggiungi le label ai punti
        texts = []
        for index, row in df_final.iterrows():
            texts.append(ax.text(x[index], y[index], row['nome_dipartimento'], ha='center', va='bottom'))

        # Ottimizza il posizionamento delle label
        adjust_text(texts)

    r = x.corr(y, method='pearson')
    rho = x.corr(y, method='spearman')
    tau = x.corr(y, method='kendall')
    y_pred = intercept + slope * x
    y_true = y
    r2 = r2_score(y_true, y_pred)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    pearson_r = Line2D([0], [0], label=f'Indice di correleazione di Pearson      r={r:.2f}', marker='s', markersize=0.01, markeredgecolor='gray',
                       markerfacecolor='gray', linestyle='')
    spearman_rho = Line2D([0], [0], label=f'Indice di correleazione di Spearman  ρ={rho:.2f}', marker='s', markersize=0.01, markeredgecolor='gray',
                          markerfacecolor='gray', linestyle='')
    kendall_tau = Line2D([0], [0], label=f'Indice di correleazione di Kendall       τ={tau:.2f}', marker='s', markersize=0.01, markeredgecolor='gray',
                         markerfacecolor='gray', linestyle='')
    determination_r2 = Line2D([0], [0], label=f'Coefficiente di determinazione         R²={r2:.2f}', marker='s', markersize=0.01, markeredgecolor='gray',
                              markerfacecolor='gray', linestyle='')
    handles.extend([pearson_r, spearman_rho, kendall_tau, determination_r2])
    plt.legend(handles=handles)
    plt.show()

In [ ]:
# Query n.1

# • I 20 dipartimenti con la maggiore percentuale di voti per Macron
#     • Primo e secondo turno


print('Primo turno')
lista_dipartimenti_1_turno_max_macron = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_1_turno, cognome='MACRON')

print('Secondo turno')
lista_dipartimenti_2_turno_max_macron = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_2_turno, cognome='MACRON')

In [ ]:
# Query n.2

# I 20 dipartimenti con la minore percentuale di voti per Macron
#     • Primo e secondo turno


print('Primo turno')
lista_dipartimenti_1_turno_min_macron = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_1_turno, cognome='MACRON', min=True)

print('Secondo turno')
lista_dipartimenti_2_turno_min_macron = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_2_turno, cognome='MACRON', min=True)

In [ ]:
# Query n.3

# I 20 dipartimenti con la maggiore percentuale di voti per Marine Le Pen
#    • Primo e secondo turno

print('Primo turno')
lista_dipartimenti_1_turno_max_lepen = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_1_turno, cognome = 'LE PEN')

print('Secondo turno')
lista_dipartimenti_2_turno_max_lepen = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_2_turno, cognome = 'LE PEN')

In [ ]:
# Query n.4

# I 20 dipartimenti con la minore percentuale di voti per Marine Le Pen
#     • Primo e secondo turno

print('Primo turno')
lista_dipartimenti_1_turno_min_lepen = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_1_turno, cognome = 'LE PEN', min = True)

print('Secondo turno')
lista_dipartimenti_2_turno_min_lepen = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_2_turno, cognome = 'LE PEN', min = True)

In [ ]:
# Query n.5

# I 20 dipartimenti con la maggiore percentuale di voti per Jean-Luc Melenchon
#     • Primo turno

print('Primo turno')
lista_dipartimenti_1_turno_max_melanchon = get_dipartimenti_min_max_perc_voti(df_risultati_elezioni_presidenziali_1_turno, cognome = 'MÉLENCHON')

In [ ]:
# Query n.6 e n.7

# I 20 dipartimenti con la maggiore e la minore disoccupazione

#seleziono il primo trimestre del 2022 e sommo il numero di persone in cerca di lavoro
primo_trimestre = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter == 1)]
primo_trimestre_sum = primo_trimestre.groupby('nome_dipartimento')['numero_medio_persone_cercanti_lavoro'].sum()

#seleziono i restanti trimestri del 2022 e sommo la differenza delle persone in cerca di lavoro rispetto al trimestre precedente
trimestri_successivi = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter > 1)]
trimestri_successivi_sum = trimestri_successivi.groupby('nome_dipartimento')['differenza_con_cercanti_lavoro_trimestre_prec'].sum()

#unisco i dati e sommo il numero di disoccupati al primo trimeste con la somma delle differenze
risultato = pd.merge(primo_trimestre_sum, trimestri_successivi_sum, on='nome_dipartimento')
risultato['disoccupazione_totale'] = risultato['numero_medio_persone_cercanti_lavoro'] + risultato['differenza_con_cercanti_lavoro_trimestre_prec']

#seleziono i 20 dipartimenti con la maggiore e minore disoccupazione totale
risultato_top_20 = risultato.nlargest(20, 'disoccupazione_totale')
risultato_bottom_20 = risultato.nsmallest(20, 'disoccupazione_totale')
risultato_top_20.reset_index(inplace=True)
risultato_bottom_20.reset_index(inplace=True)

lista_risultato_top_20 = risultato_top_20["nome_dipartimento"].values.tolist()
lista_risultato_bottom_20 = risultato_bottom_20["nome_dipartimento"].values.tolist()

print("I 20 dipartimenti con la maggiore disoccupazione")
display(lista_risultato_top_20)
print("I 20 dipartimenti con la minore disoccupazione")
display(lista_risultato_bottom_20)

In [ ]:
# Query n.8 e n.9

# I 20 dipartimenti con il maggiore e il minore reddito pro capite

# ordina i dipartimenti per reddito descrescente
df_final = df_reddito_dipartimenti.sort_values(by="reddito_pro_capite", ascending=False)[['nome_dipartimento', 'reddito_pro_capite']]
df_final = df_final[:20]
df_final.reset_index(drop=True, inplace=True)

# stampa i 20 dipartimenti con il maggiore reddito pro capite
lista_df_final = df_final["nome_dipartimento"].values.tolist()
print("I 20 dipartimenti con il maggiore reddito pro capite")
display(lista_df_final)

# ordina i dipartimenti per reddito crescente
df_final_2 = df_reddito_dipartimenti.sort_values(by="reddito_pro_capite", ascending=True)[['nome_dipartimento', 'reddito_pro_capite']]
df_final_2 = df_final_2[:20]
df_final_2.reset_index(drop=True, inplace=True)

# stampa i 20 dipartimenti con il minore reddito pro capite
lista_df_final_2 = df_final_2["nome_dipartimento"].values.tolist()
print("I 20 dipartimenti con il minore reddito pro capite")
display(lista_df_final_2)

In [ ]:
# Query n.10 e n.11

# I 20 dipartimenti con la maggiore e minore percentuale di vaccinazioni COVID

# seleziona i dati dei vaccini somministrati a tutte le età
dati_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']

# ottiene la copertura vaccinale (relativa ad un ciclo vaccinale completo, almeno seconda dose) più recente per dipartimento 
df_copertura_dipartimento = dati_tutte_eta.groupby(['nome_dipartimento'], as_index=False)['copertura_vaccinale_seconda_dose'].max()

# ordina per copertura vaccinale descrescente
df_final = df_copertura_dipartimento.sort_values(by="copertura_vaccinale_seconda_dose",ascending=False)
df_final = df_final[:20]
df_final.reset_index(drop=True, inplace=True)

# stampa i 20 dipartimenti con la maggiore percentuale di vaccinazioni COVID
lista_df_final = df_final["nome_dipartimento"].values.tolist()
print("I 20 dipartimenti con la maggiore percentuale di vaccinazioni COVID")
display(lista_df_final)

# ordina per copertura vaccinale crescente
df_final_2 = df_copertura_dipartimento.sort_values(by="copertura_vaccinale_seconda_dose",ascending=True)
df_final_2 = df_final_2[:20]
df_final_2.reset_index(drop=True, inplace=True)

# stampa i 20 dipartimenti con la minore percentuale di vaccinazioni COVID
lista_df_final_2 = df_final_2["nome_dipartimento"].values.tolist()
print("I 20 dipartimenti con la minore percentuale di vaccinazioni COVID")
display(lista_df_final_2)

In [ ]:
# Query n.12 e n.13

# 10 dipartimenti con la maggiore inflazione e 10 dipartimenti con la minore inflazione

# ordina il dataframe per indice di prezzo decrescente
df_inflazione.sort_values(by='indice_prezzo', ascending=False)
#display(df_inflazione)

# ottengo e stampo i 10 dipartimenti con inflazione maggiore
lista_dipartimenti_inflazione_maggiore = df_inflazione['nome_dipartimento'].values.tolist()[:10]
print('Dipartimenti con la maggiore inflazione')
display(lista_dipartimenti_inflazione_maggiore)

# ottengo e stampo i 10 dipartimenti con inflazione minore
lista_dipartimenti_inflazione_minore = df_inflazione['nome_dipartimento'].values.tolist()[-10:]
print('Dipartimenti con la minore inflazione')
display(lista_dipartimenti_inflazione_minore)

In [ ]:
# Query n.14

# 10 dipartimenti con il maggior cambiamento di voti tra il primo turno delle presidenziali e il primo turno delle legislative

# calcolo e stampo per i vari candidati e i relativi partiti
print('Macron - Ensamble')
lista_dipartimenti_differenza_macron_ensamble = differenza_presidenziali_legislative(['MACRON'], 'ENS')

print('Le Pen - RN')
lista_dipartimenti_differenza_lepen_rn = differenza_presidenziali_legislative(['LE PEN'], 'RN')

print('Melenchon, Jadot, Hidalgo - Nupes')
lista_dipartimenti_differenza_ = differenza_presidenziali_legislative(['MELENCHON', 'JADOT', 'HIDALGO'], 'NUP')

In [ ]:
# Correlazione n.1
# Esiste una correlazione, in un dato dipartimento, tra voto a Macron nel primo turno e reddito pro capite?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'MACRON']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_macron = somma_voti_espressi.copy(deep=True)
df_voti_macron.reset_index(drop=True, inplace=True)
#display(df_voti_macron)

df_reddito = df_reddito_dipartimenti.copy(deep=True)
df_reddito = df_reddito.astype({"reddito_pro_capite": float})
df_reddito.reset_index(drop=True, inplace=True)
#display(df_reddito)

df_final = pd.merge(df_voti_macron, df_reddito, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['reddito_pro_capite']

correlation_calculator(x, y, '% voti a favore/voti totali per Macron', 'reddito pro capite')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.66.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.6 e 0.79 è forte, quindi esiste una correlazione forte, nei vari dipartimenti, tra voto a Macron nel primo turno e reddito pro capite.')

In [ ]:
# Correlazione n.2
# Esiste una correlazione, in un dato dipartimento, tra voto a Macron nel primo turno e disoccupazione?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'MACRON']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_macron = somma_voti_espressi.copy(deep=True)
df_voti_macron.reset_index(drop=True, inplace=True)
#display(df_voti_macron)

primo_trimestre = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter == 1)]
primo_trimestre_sum = primo_trimestre.groupby('nome_dipartimento')['numero_medio_persone_cercanti_lavoro'].sum()
trimestri_successivi = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter > 1)]
trimestri_successivi_sum = trimestri_successivi.groupby('nome_dipartimento')['differenza_con_cercanti_lavoro_trimestre_prec'].sum()
risultato = pd.merge(primo_trimestre_sum, trimestri_successivi_sum, on='nome_dipartimento')
risultato['disoccupazione_totale'] = risultato['numero_medio_persone_cercanti_lavoro'] + risultato['differenza_con_cercanti_lavoro_trimestre_prec']
df_final_disoccupazione = risultato.reset_index()
#display(df_final_disoccupazione)

df_final = pd.merge(df_voti_macron, df_final_disoccupazione, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['disoccupazione_totale']

correlation_calculator(x, y, '% voti a favore/voti totali per Macron', 'numero disoccupati')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.21.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.2 e 0.39 è debole, quindi esiste una correlazione debole, nei vari dipartimenti, tra voto a Macron nel primo turno e numero di disoccupati.')

In [ ]:
# Correlazione n.3
# Esiste una correlazione, in un dato dipartimento, tra voto a Macron nel primo turno e tasso di vaccinazione?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'MACRON']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_macron = somma_voti_espressi.copy(deep=True)
df_voti_macron.reset_index(drop=True, inplace=True)
#display(df_voti_macron)

dati_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']
df_copertura_dipartimento = dati_tutte_eta.groupby(['nome_dipartimento'], as_index=False)['copertura_vaccinale_seconda_dose'].max()
#display(df_copertura_dipartimento)

df_final = pd.merge(df_voti_macron, df_copertura_dipartimento, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['copertura_vaccinale_seconda_dose']

correlation_calculator(x, y, '% voti a favore/voti totali per Macron', 'tasso di vaccinazione')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.39.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.2 e 0.39 è debole, quindi esiste una correlazione debole, nei vari dipartimenti, tra voto a Macron nel primo turno e vaccinazioni.')

In [ ]:
# Correlazione n.4
# Esiste una correlazione, in un dato dipartimento, tra voto a Le Pen nel primo turno e reddito dichiarato pro capite?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'LE PEN']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_lepen = somma_voti_espressi.copy(deep=True)
df_voti_lepen.reset_index(drop=True, inplace=True)
#display(df_voti_lepen)

df_reddito = df_reddito_dipartimenti.copy(deep=True)
df_reddito = df_reddito.astype({"reddito_pro_capite": float})
df_reddito.reset_index(drop=True, inplace=True)
#display(df_reddito)

df_final = pd.merge(df_voti_lepen, df_reddito, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['reddito_pro_capite']

correlation_calculator(x, y, '% voti a favore/voti totali per Le Pen', 'reddito pro capite')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a -0.73.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.6 e 0.79 è forte, quindi esiste una correlazione forte, nei vari dipartimenti, tra voto a Le Pen nel primo turno e reddito pro capite.')

In [ ]:
# Correlazione n.5
# Esiste una correlazione, in un dato dipartimento, tra voto a Le Pen nel primo turno e disoccupazione?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'LE PEN']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_lepen = somma_voti_espressi.copy(deep=True)
df_voti_lepen.reset_index(drop=True, inplace=True)
#display(df_voti_lepen)

primo_trimestre = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter == 1)]
primo_trimestre_sum = primo_trimestre.groupby('nome_dipartimento')['numero_medio_persone_cercanti_lavoro'].sum()
trimestri_successivi = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter > 1)]
trimestri_successivi_sum = trimestri_successivi.groupby('nome_dipartimento')['differenza_con_cercanti_lavoro_trimestre_prec'].sum()
risultato = pd.merge(primo_trimestre_sum, trimestri_successivi_sum, on='nome_dipartimento')
risultato['disoccupazione_totale'] = risultato['numero_medio_persone_cercanti_lavoro'] + risultato['differenza_con_cercanti_lavoro_trimestre_prec']
df_final_disoccupazione = risultato.reset_index()
#display(df_final_disoccupazione)

df_final = pd.merge(df_voti_lepen, df_final_disoccupazione, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['disoccupazione_totale']

correlation_calculator(x, y, '% voti a favore/voti totali per Le Pen', 'numero disoccupati')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a -0.51.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.4 e 0.59 è moderata, quindi esiste una correlazione moderata, nei vari dipartimenti, tra voto a Le Pen nel primo turno e numero di disoccupati.')

In [ ]:
# Correlazione n.6
# Esiste una correlazione, in un dato dipartimento, tra voto a Le Pen nel primo turno e tasso di vaccinazione?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'LE PEN']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_lepen = somma_voti_espressi.copy(deep=True)
df_voti_lepen.reset_index(drop=True, inplace=True)
#display(df_voti_lepen)

dati_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']
df_copertura_dipartimento = dati_tutte_eta.groupby(['nome_dipartimento'], as_index=False)['copertura_vaccinale_seconda_dose'].max()
#display(df_copertura_dipartimento)

df_final = pd.merge(df_voti_lepen, df_copertura_dipartimento, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['copertura_vaccinale_seconda_dose']

correlation_calculator(x, y, '% voti a favore/voti totali per Le Pen', 'tasso di vaccinazione')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.25.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.2 e 0.39 è debole, quindi esiste una correlazione debole, nei vari dipartimenti, tra voto a Le Pen nel primo turno e vaccinazioni.')

In [ ]:
# Correlazione n.7
# Esiste una correlazione, in un dato dipartimento, tra voto a Melenchon nel primo turno e reddito dichiarato pro capite?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'MÉLENCHON']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_melenchon = somma_voti_espressi.copy(deep=True)
df_voti_melenchon.reset_index(drop=True, inplace=True)
#display(df_voti_melenchon)

df_reddito = df_reddito_dipartimenti.copy(deep=True)
df_reddito = df_reddito.astype({"reddito_pro_capite": float})
df_reddito.reset_index(drop=True, inplace=True)
#display(df_reddito)

df_final = pd.merge(df_voti_melenchon, df_reddito, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['reddito_pro_capite']

correlation_calculator(x, y, '% voti a favore/voti totali per Mélenchon', 'reddito pro capite')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.35.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.2 e 0.39 è debole, quindi esiste una correlazione debole, nei vari dipartimenti, tra voto a Mélenchon nel primo turno e reddito pro capite.')

In [ ]:
# Correlazione n.8
# Esiste una correlazione, in un dato dipartimento, tra voto a Melenchon nel primo turno e disoccupazione?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'MÉLENCHON']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_melenchon = somma_voti_espressi.copy(deep=True)
df_voti_melenchon.reset_index(drop=True, inplace=True)
#display(df_voti_melenchon)

primo_trimestre = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter == 1)]
primo_trimestre_sum = primo_trimestre.groupby('nome_dipartimento')['numero_medio_persone_cercanti_lavoro'].sum()
trimestri_successivi = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter > 1)]
trimestri_successivi_sum = trimestri_successivi.groupby('nome_dipartimento')['differenza_con_cercanti_lavoro_trimestre_prec'].sum()
risultato = pd.merge(primo_trimestre_sum, trimestri_successivi_sum, on='nome_dipartimento')
risultato['disoccupazione_totale'] = risultato['numero_medio_persone_cercanti_lavoro'] + risultato['differenza_con_cercanti_lavoro_trimestre_prec']
df_final_disoccupazione = risultato.reset_index()
#display(df_final_disoccupazione)

df_final = pd.merge(df_voti_melenchon, df_final_disoccupazione, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['disoccupazione_totale']

correlation_calculator(x, y, '% voti a favore/voti totali per Mélenchon', 'numero disoccupati')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.65.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.6 e 0.79 è forte, quindi esiste una correlazione forte, nei vari dipartimenti, tra voto a Mélenchon nel primo turno e numero di disoccupati.')

In [ ]:
# Correlazione n.9
# Esiste una correlazione, in un dato dipartimento, tra voto a Melenchon nel primo turno e tasso di vaccinazione?

somma_voti_espressi = df_risultati_elezioni_presidenziali_1_turno.groupby(['nome_dipartimento', 'cognome'], as_index=False)[['voti','espressi']].sum()
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['cognome'] == 'MÉLENCHON']
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_melenchon = somma_voti_espressi.copy(deep=True)
df_voti_melenchon.reset_index(drop=True, inplace=True)
#display(df_voti_melenchon)

dati_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']
df_copertura_dipartimento = dati_tutte_eta.groupby(['nome_dipartimento'], as_index=False)['copertura_vaccinale_seconda_dose'].max()
#display(df_copertura_dipartimento)

df_final = pd.merge(df_voti_melenchon, df_copertura_dipartimento, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['copertura_vaccinale_seconda_dose']

correlation_calculator(x, y, '% voti a favore/voti totali per Melenchon', 'tasso di vaccinazione')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a -0.64.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nTra 0.6 e 0.79 è forte, quindi esiste una correlazione forte, nei vari dipartimenti, tra voto a Melenchon nel primo turno e vaccinazioni.')

In [ ]:
# Correlazione n.10
# Esiste una correlazione, in un dato dipartimento, tra voto a Ensemble/RN/Nupes/LR nel primo turno delle legislative e reddito pro capite?
somma_voti_espressi = df_risultati_elezioni_legislative_1_turno.groupby(['nome_dipartimento', 'partito'], as_index=False)[['voti','espressi']].sum()
partiti_selezionati = ['ENS', 'RN', 'NUP', 'LR']
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['partito'].isin(partiti_selezionati)]
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_gruppo = somma_voti_espressi.copy(deep=True)
df_voti_gruppo.reset_index(drop=True, inplace=True)
#display(df_voti_gruppo)

df_reddito = df_reddito_dipartimenti.copy(deep=True)
df_reddito = df_reddito.astype({"reddito_pro_capite": float})
df_reddito.reset_index(drop=True, inplace=True)
#display(df_reddito)

df_final = pd.merge(df_voti_gruppo, df_reddito, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['reddito_pro_capite']

correlation_calculator(x, y, '% voti a favore/voti totali per Ensemble/RN/Nupes/LR', 'reddito pro capite')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a -0.002.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nUn valore vicino a 0 indica una correlazione debole o nulla quindi non esiste una correlazione, nei vari dipartimenti, tra voto a Ensemble/RN/Nupes/LR nel primo turno e reddito pro capite.')

In [ ]:
# Correlazione n.11
# Esiste una correlazione, in un dato dipartimento, tra voto a Ensemble/RN/Nupes/LR nel primo turno delle legislative e disoccupazione?
somma_voti_espressi = df_risultati_elezioni_legislative_1_turno.groupby(['nome_dipartimento', 'partito'], as_index=False)[['voti','espressi']].sum()
partiti_selezionati = ['ENS', 'RN', 'NUP', 'LR']
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['partito'].isin(partiti_selezionati)]
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_gruppo = somma_voti_espressi.copy(deep=True)
df_voti_gruppo.reset_index(drop=True, inplace=True)
#display(df_voti_gruppo)

primo_trimestre = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter == 1)]
primo_trimestre_sum = primo_trimestre.groupby('nome_dipartimento')['numero_medio_persone_cercanti_lavoro'].sum()
trimestri_successivi = df_disoccupazione[(pd.to_datetime(df_disoccupazione['data']).dt.year == 2022 ) & (pd.to_datetime(df_disoccupazione['data']).dt.quarter > 1)]
trimestri_successivi_sum = trimestri_successivi.groupby('nome_dipartimento')['differenza_con_cercanti_lavoro_trimestre_prec'].sum()
risultato = pd.merge(primo_trimestre_sum, trimestri_successivi_sum, on='nome_dipartimento')
risultato['disoccupazione_totale'] = risultato['numero_medio_persone_cercanti_lavoro'] + risultato['differenza_con_cercanti_lavoro_trimestre_prec']
df_final_disoccupazione = risultato.reset_index()
#display(df_final_disoccupazione)

df_final = pd.merge(df_voti_gruppo, df_final_disoccupazione, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['disoccupazione_totale']

correlation_calculator(x, y, '% voti a favore/voti totali per Ensemble/RN/Nupes/LR', 'tasso disoccupazione')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a -0.03.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nUn valore vicino a 0 indica una correlazione debole o nulla quindi non esiste una correlazione, nei vari dipartimenti, tra voto a Ensemble/RN/Nupes/LR nel primo turno e disoccupazione.')

In [ ]:
# Correlazione n.12
# Esiste una correlazione, in un dato dipartimento, tra voto a Ensemble/RN/Nupes/LR nel primo turno delle legislative e tasso di vaccinazione?
somma_voti_espressi = df_risultati_elezioni_legislative_1_turno.groupby(['nome_dipartimento', 'partito'], as_index=False)[['voti','espressi']].sum()
partiti_selezionati = ['ENS', 'RN', 'NUP', 'LR']
somma_voti_espressi = somma_voti_espressi[somma_voti_espressi['partito'].isin(partiti_selezionati)]
somma_voti_espressi['% voti_a_favore/voti_totali'] = somma_voti_espressi.apply(percentage_calculator, axis=1)
df_voti_gruppo = somma_voti_espressi.copy(deep=True)
df_voti_gruppo.reset_index(drop=True, inplace=True)
#display(df_voti_gruppo)

dati_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']
df_copertura_dipartimento = dati_tutte_eta.groupby(['nome_dipartimento'], as_index=False)['copertura_vaccinale_seconda_dose'].max()
#display(df_copertura_dipartimento)

df_final = pd.merge(df_voti_gruppo, df_copertura_dipartimento, on='nome_dipartimento')
#display(df_final)

x = df_final['% voti_a_favore/voti_totali']
y = df_final['copertura_vaccinale_seconda_dose']

correlation_calculator(x, y, '% voti a favore/voti totali per Ensemble/RN/Nupes/LR', 'vaccinazioni')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.02.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nUn valore vicino a 0 indica una correlazione debole o nulla quindi non esiste una correlazione, nei vari dipartimenti, tra voto a Ensemble/RN/Nupes/LR nel primo turno e vaccinazioni.')

In [ ]:
# Correlazione n.13
# Esiste una correlazione, in un dato dipartimento, tra tasso di vaccinazione e reddito dichiarato pro capite?

vaccinazioni_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']
df_tasso_vaccinazione = vaccinazioni_tutte_eta.groupby(['nome_dipartimento'], as_index=False)[['copertura_vaccinale_seconda_dose']].max()
#display(df_media_tasso_vaccinazione)

df_reddito_pro_capite = df_reddito_dipartimenti[['nome_dipartimento', 'reddito_pro_capite']]
#display(df_media_reddito)

df_final = pd.merge(df_tasso_vaccinazione, df_reddito_pro_capite, on='nome_dipartimento')
#display(df_final)

x = df_final['copertura_vaccinale_seconda_dose']
y = df_final['reddito_pro_capite']

correlation_calculator(x, y, 'tasso di vaccinazione', 'reddito pro capite')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.14.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nUn valore vicino a 0 indica una correlazione debole o nulla quindi non esiste una correlazione, nei vari dipartimenti, tra tasso di vaccinazione e reddito pro capite.')

In [ ]:
# Correlazione n.14
# Esiste una correlazione, in un dato dipartimento, tra tasso di vaccinazione e tasso di disoccupazione?
vaccinazioni_tutte_eta = df_vaccinazioni_covid[df_vaccinazioni_covid['etichetta_variabile'] == 'Tous âges']
df_tasso_vaccinazione = vaccinazioni_tutte_eta.groupby(['nome_dipartimento'], as_index=False)[['copertura_vaccinale_seconda_dose']].max()
#display(df_media_tasso_vaccinazione)

df_media_disoccupazione = df_disoccupazione.groupby(['nome_dipartimento'], as_index=False)['numero_medio_persone_cercanti_lavoro'].mean();
#display(df_media_reddito)

df_final = pd.merge(df_tasso_vaccinazione, df_media_disoccupazione, on='nome_dipartimento')
#display(df_final)

x = df_final['copertura_vaccinale_seconda_dose']
y = df_final['numero_medio_persone_cercanti_lavoro']

correlation_calculator(x, y, 'tasso di vaccinazione', 'disoccupazione')

print('Il valore del coefficiente che fa capo alla correlazione lineare ovvero il coefficiente di Pearson è pari a 0.28.\
       \nSecondo Evans la forza della correlazione lineare viene decisa in base al valore assoluto di tale coefficiente. \
       \nUn valore vicino a 0.2 e 0.39 indica una correlazione debole quindi non esiste una correlazione, nei vari dipartimenti, tra tasso di vaccinazione e tasso di dispccupazione.')

In [ ]:
# Esistono altre correlazioni, anche spurie?
